In [ ]:
# Bagging and Random Forest

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
random.seed(1000)

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated
print("Hello World")
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

Hello World
---Run time is 0.0005242000000009739 seconds ---



In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
df2017 = pd.read_csv('SRER_2017_training_bi.csv').dropna()
df2021 = pd.read_csv("SRER21_dataset_v1.csv").dropna()
df2017["Year"] = '2017'
df2021["Year"] = '2021'
frames = [df2017,df2021]
dfCombined = pd.concat(frames)
dfCombined = dfCombined.reindex(columns=['OID_','Id', 'gridcode','Shape_Length','Shape_Area', 'CH_mean', 'ARVI_max', 'ARVI_mean', 'ARVI_med', 'EVI_max', 'EVI_mean', 'EVI_med', 'NDVI_max', 'NDVI_mean', 'NDVI_med', 'SAVI_max', 'SAVI_mean', 'SAVI_med', 'Year', 'Veg_class'])



dfJornada2017 = pd.read_csv("JORN17_dataset_v1.csv", na_values = '?').dropna()
dfJornada2021 = pd.read_csv("JORN21_dataset_v1.csv", na_values = '?').dropna()

dfJornada2021

,OID_,Id,gridcode,Shape_Length,Shape_Area,CH_mean,ARVI_mean,ARVI_med,ARVI_max,EVI_mean,EVI_med,EVI_max,NDVI_mean,NDVI_med,NDVI_max,SAVI_mean,SAVI_med,SAVI_max,Veg_class
0,3,3,3,11.2,2.43,0.013333,0.119338,0.090758,0.237412,0.210226,0.187676,0.270899,0.304648,0.279301,0.393630,0.214770,0.193736,0.265742,woody
1,4,4,4,10.2,2.75,0.123333,0.265313,0.206095,0.404770,0.284236,0.253169,0.350252,0.421242,0.374927,0.522831,0.280641,0.257630,0.333867,woody
2,5,5,5,14.0,4.60,0.015000,0.362829,0.350086,0.464428,0.307055,0.305663,0.361261,0.485779,0.477382,0.560481,0.296065,0.295391,0.340641,woody
3,6,6,6,16.8,6.35,0.014000,0.277826,0.303609,0.391677,0.270045,0.260203,0.310201,0.425165,0.436553,0.501630,0.267132,0.264301,0.296636,woody
4,7,7,7,12.2,3.04,0.010000,0.135110,0.135110,0.160633,0.230423,0.230423,0.253389,0.329082,0.329082,0.351473,0.239081,0.239081,0.260661,woody
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,2278,2278,2278,25.2,6.63,0.015000,0.128259,0.134121,0.167685,0.237659,0.245145,0.256763,0.334798,0.337580,0.365527,0.252164,0.257355,0.268680,non-woody
2227,2279,2279,2279,15.2,5.33,0.134000,0.485618,0.489819,0.519026,0.416102,0.438885,0.441772,0.583201,0.585438,0.603520,0.386171,0.403764,0.410255,woody
2228,2280,2280,2280,17.0,4.98,0.114000,0.313633,0.318283,0.474006,0.350407,0.341923,0.436805,0.483337,0.490012,0.591361,0.350956,0.346441,0.412744,non-woody
2229,2281,2281,2281,19.8,6.49,0.011667,0.119920,0.108993,0.198284,0.230958,0.227932,0.271254,0.323523,0.317326,0.384514,0.243325,0.242532,0.278123,non-woody


In [6]:
df2017.Veg_class = df2017.Veg_class.map({'non-woody':0, 'woody':1})
df2021.Veg_class = df2021.Veg_class.map({'non-woody':0, 'woody':1})
dfJornada2017.Veg_class = dfJornada2017.Veg_class.map({'non-woody':0, 'woody':1})
dfJornada2021.Veg_class = dfJornada2021.Veg_class.map({'non-woody':0, 'woody':1})


df2017

,OID_,Id,gridcode,Shape_Length,Shape_Area,CH_mean,ARVI_max,ARVI_mean,ARVI_med,EVI_max,EVI_mean,EVI_med,NDVI_max,NDVI_mean,NDVI_med,SAVI_max,SAVI_mean,SAVI_med,Veg_class,Year
0,1,2,2,41.6,17.41,0.190714,0.609099,0.344183,0.379428,0.443455,0.292047,0.290985,0.665698,0.476709,0.507514,0.406991,0.286203,0.288351,1,2017
1,2,3,3,31.4,5.05,0.045000,0.387755,0.245301,0.274936,0.332059,0.252919,0.244477,0.526375,0.406745,0.433791,0.328585,0.254814,0.248126,0,2017
2,3,5,5,33.4,4.84,0.010000,0.038844,0.004040,0.026805,0.150550,0.131985,0.134891,0.208300,0.181575,0.205117,0.150184,0.131642,0.135063,0,2017
3,4,6,6,32.2,14.20,0.011667,0.189842,0.033418,0.018961,0.198972,0.141579,0.138338,0.341480,0.207390,0.197327,0.197330,0.141500,0.138725,0,2017
4,5,7,7,28.8,10.73,0.013750,0.322694,0.080315,0.061453,0.248442,0.158710,0.142326,0.457103,0.247391,0.228112,0.248598,0.158618,0.142817,0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,4335,4487,4487,8.8,2.30,0.020000,0.290182,0.290182,0.290182,0.283959,0.283959,0.283959,0.419200,0.419200,0.419200,0.269486,0.269486,0.269486,1,2017
4335,4336,4488,4488,10.8,2.03,0.020000,0.408320,0.408320,0.408320,0.372648,0.372648,0.372648,0.510469,0.510469,0.510469,0.340879,0.340879,0.340879,1,2017
4336,4337,4489,4489,11.8,2.62,0.015000,0.437195,0.413297,0.413297,0.235606,0.234926,0.234926,0.499074,0.485601,0.485601,0.227606,0.226722,0.226722,1,2017
4337,4338,4490,4490,11.2,3.09,0.015000,0.570094,0.550652,0.550652,0.416298,0.388085,0.388085,0.640722,0.621694,0.621694,0.387576,0.363288,0.363288,1,2017


In [7]:
# Bagging and Random Forest

In [8]:
#Bagging 2017 -> 2021 SRER 

In [ ]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = RandomForestClassifier(max_features = 10, random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2021.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2021.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score